### Import Dependencies


In [33]:
import pandas as pd
import json
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect, desc
import sqlite3

### Import SQLite Database and Find Table Names

In [3]:
conn = sqlite3.connect("Election2020Data_ew.sqlite")
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

# engine = create_engine("sqlite:///Election2020Data_ew.sqlite")


[('2020Donors',)]


### Display Limited Dataframe

In [4]:
df_500K = pd.read_sql_query("select * from '2020Donors' limit 500000;", conn)
df_500K

,Record_id,Flag_Orgind,Org_name,Last_Name,First_Name,Middle_Name,Prefix,Suffix,Address_One,Address_Two,City,State,Zip,Employer,Occupation,Amount,Date,Aggregate_Amount,Cycle,Campaign
0,1,IND,None,Drake,Russell,None,None,None,2325 Highland Cres S,None,Birmingham,AL,35205,None,Retired,250.0,2019-03-31,250.00,2020,Kamala Harris For The People
1,2,IND,None,Dunn,Cody,None,None,None,7280 Lakewood Rd,None,Mobile,AL,36695,Wal-Mart,Clerk,100.0,2019-03-31,250.00,2020,Kamala Harris For The People
2,3,IND,None,McPhillips,Frank,None,None,None,3715 Old Leeds Rd,None,Mountain Brk,AL,35213,None,Retired,500.0,2019-03-31,500.00,2020,Kamala Harris For The People
3,4,IND,None,Rumble,Henry,None,None,None,3779 Crossings Crst,None,Birmingham,AL,35242,Self-Employed,PT Consultant,25.0,2019-03-30,297.84,2020,Kamala Harris For The People
4,5,IND,None,Rumble,Henry,None,None,None,3779 Crossings Crst,None,Birmingham,AL,35242,Self-Employed,PT Consultant,25.0,2019-03-28,297.84,2020,Kamala Harris For The People
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,499996,IND,None,Paul,David,None,None,None,3690 Shannon Rd,None,Los Angeles,CA,90027.0,Self Employed,Investor,1000.0,2019-06-30,1050.00,2020,Cory 2020
499996,499997,IND,None,Gramling,Lyle,None,None,None,1248 Commons Dr,None,Sacramento,CA,95825.0,Not Employed,Not Employed,25.0,2019-04-27,283.30,2020,Cory 2020
499997,499998,IND,None,Miller,Terri,None,None,None,9100 Wilshire Blvd,Ste 1000,Beverly Hills,CA,90212.0,Self-Employed,Producer,500.0,2019-05-17,500.00,2020,Cory 2020
499998,499999,IND,None,Lauder,Laura,H.,None,None,88 Mercedes Ln,None,Atherton,CA,94027.0,Lauder Partners LLC,Venture Capitalist,100.0,2019-06-14,2800.00,2020,Cory 2020


### Find Total Number of Rows

In [5]:
cursor.execute("SELECT COUNT(*) from '2020Donors'")
cur_result = cursor.fetchone()
print(cur_result)

(9763877,)


### Create Dataframe Subsets Using Individual Columns

In [19]:
last_names = pd.read_sql_query("select Last_Name from '2020Donors';", conn)
first_names = pd.read_sql_query("select First_Name from '2020Donors';", conn)
address_one = pd.read_sql_query("select Address_One from '2020Donors';", conn)

In [14]:
city = pd.read_sql_query("select City from '2020Donors';", conn)
state = pd.read_sql_query("select State from '2020Donors';", conn)
zip_code = pd.read_sql_query("select Zip from '2020Donors';", conn)

In [20]:
employer = pd.read_sql_query("select Employer from '2020Donors';", conn)
occupation = pd.read_sql_query("select Occupation from '2020Donors';", conn)
amount = pd.read_sql_query("select Amount from '2020Donors';", conn)

In [27]:
date = pd.read_sql_query("select Date from '2020Donors';", conn)
aggregate_amount = pd.read_sql_query("select Aggregate_Amount from '2020Donors';", conn)
campaign = pd.read_sql_query("select Campaign from '2020Donors';", conn)

In [22]:
name_address_df = pd.concat([last_names, first_names, address_one], axis = 1)
name_address_df.head()


,Last_Name,First_Name,Address_One
0,Drake,Russell,2325 Highland Cres S
1,Dunn,Cody,7280 Lakewood Rd
2,McPhillips,Frank,3715 Old Leeds Rd
3,Rumble,Henry,3779 Crossings Crst
4,Rumble,Henry,3779 Crossings Crst


In [23]:
city_state_zip_df = pd.concat([city, state, zip_code], axis = 1)
city_state_zip_df.head()

,City,State,Zip
0,Birmingham,AL,35205
1,Mobile,AL,36695
2,Mountain Brk,AL,35213
3,Birmingham,AL,35242
4,Birmingham,AL,35242


In [24]:
employer_occupation_amount_df = pd.concat([employer, occupation, amount], axis = 1)
employer_occupation_amount_df.head()

,Employer,Occupation,Amount
0,None,Retired,250.0
1,Wal-Mart,Clerk,100.0
2,None,Retired,500.0
3,Self-Employed,PT Consultant,25.0
4,Self-Employed,PT Consultant,25.0


In [28]:
date_agg_campaign_df = pd.concat([date, aggregate_amount, campaign], axis = 1)
date_agg_campaign_df.head()

,Date,Aggregate_Amount,Campaign
0,2019-03-31,250.00,Kamala Harris For The People
1,2019-03-31,250.00,Kamala Harris For The People
2,2019-03-31,500.00,Kamala Harris For The People
3,2019-03-30,297.84,Kamala Harris For The People
4,2019-03-28,297.84,Kamala Harris For The People


### Recombine Dataframes

In [29]:
final_df = pd.concat([name_address_df, city_state_zip_df, employer_occupation_amount_df, date_agg_campaign_df], axis = 1)
final_df.head()

,Last_Name,First_Name,Address_One,City,State,Zip,Employer,Occupation,Amount,Date,Aggregate_Amount,Campaign
0,Drake,Russell,2325 Highland Cres S,Birmingham,AL,35205,None,Retired,250.0,2019-03-31,250.00,Kamala Harris For The People
1,Dunn,Cody,7280 Lakewood Rd,Mobile,AL,36695,Wal-Mart,Clerk,100.0,2019-03-31,250.00,Kamala Harris For The People
2,McPhillips,Frank,3715 Old Leeds Rd,Mountain Brk,AL,35213,None,Retired,500.0,2019-03-31,500.00,Kamala Harris For The People
3,Rumble,Henry,3779 Crossings Crst,Birmingham,AL,35242,Self-Employed,PT Consultant,25.0,2019-03-30,297.84,Kamala Harris For The People
4,Rumble,Henry,3779 Crossings Crst,Birmingham,AL,35242,Self-Employed,PT Consultant,25.0,2019-03-28,297.84,Kamala Harris For The People


### Convert Dates to Datetime

In [30]:
final_df["Date"] = pd.to_datetime(final_df["Date"], format = "%Y-%m-%d")
final_df.dtypes


Last_Name                   object
First_Name                  object
Address_One                 object
City                        object
State                       object
Zip                         object
Employer                    object
Occupation                  object
Amount                     float64
Date                datetime64[ns]
Aggregate_Amount           float64
Campaign                    object
dtype: object

### Confirm Final Rowcount

In [32]:
len(final_df)

9763877

In [34]:
final_df.to_json("final_df.json", orient = "table")
